In [2]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense,Lambda, Activation, concatenate,ZeroPadding2D, BatchNormalization,Concatenate,GlobalAveragePooling2D, Dropout,Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  
import cv2

%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [10]:
def Conv2D_BN_AC(X, filters, kernel, strides, padding, activation):
    X = Conv2D(filters, kernel, strides=strides, padding=padding)(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation(activation)(X)
    return X

def Inception_v3(input_shape = (299,299,3)):
    X_input = Input(input_shape)
    X = Conv2D_BN_AC(X_input, 32, (3, 3), (2, 2), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 32, (3, 3), (1, 1), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 64, (3, 3), (1, 1), 'same', 'relu')
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Conv2D_BN_AC(X, 80, (3, 3), (1, 1), 'valid', 'relu')
    X = Conv2D_BN_AC(X, 192, (3, 3), (1, 1), 'same', 'relu')
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    #block mixed_5b
    mixed_5b1 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_5b2 = Conv2D_BN_AC(X, 48, (1, 1), (1, 1), 'same', 'relu')
    mixed_5b2 = Conv2D_BN_AC(mixed_5b2, 64, (5, 5), (1, 1), 'same', 'relu')
    mixed_5b3 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_5b3 = Conv2D_BN_AC(mixed_5b3, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_5b3 = Conv2D_BN_AC(mixed_5b3, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_5b4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_5b4 = Conv2D_BN_AC(mixed_5b4, 32, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_5b1,mixed_5b2,mixed_5b3,mixed_5b4], axis=-1)
    
    #block mixed_5c
    mixed_5c1 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_5c2 = Conv2D_BN_AC(X, 48, (1, 1), (1, 1), 'same', 'relu')
    mixed_5c2 = Conv2D_BN_AC(mixed_5c2, 64, (5, 5), (1, 1), 'same', 'relu')
    mixed_5c3 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_5c3 = Conv2D_BN_AC(mixed_5c3, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_5c3 = Conv2D_BN_AC(mixed_5c3, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_5c4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_5c4 = Conv2D_BN_AC(mixed_5c4, 64, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_5c1,mixed_5c2,mixed_5c3,mixed_5c4], axis=-1)
    
    #block mixed_5d
    mixed_5c1 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_5c2 = Conv2D_BN_AC(X, 48, (1, 1), (1, 1), 'same', 'relu')
    mixed_5c2 = Conv2D_BN_AC(mixed_5c2, 64, (5, 5), (1, 1), 'same', 'relu')
    mixed_5c3 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_5c3 = Conv2D_BN_AC(mixed_5c3, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_5c3 = Conv2D_BN_AC(mixed_5c3, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_5c4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_5c4 = Conv2D_BN_AC(mixed_5c4, 64, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_5c1,mixed_5c2,mixed_5c3,mixed_5c4], axis=-1)
    
    #block mixed_6a
    mixed_6a1 = Conv2D_BN_AC(X, 384, (3, 3), (2, 2), 'valid', 'relu')
    mixed_6a2 = Conv2D_BN_AC(X, 64, (1, 1), (1, 1), 'same', 'relu')
    mixed_6a2 = Conv2D_BN_AC(mixed_6a2, 96, (3, 3), (1, 1), 'same', 'relu')
    mixed_6a2 = Conv2D_BN_AC(mixed_6a2, 96, (3, 3), (2, 2), 'valid', 'relu')
    mixed_6a3 = MaxPooling2D((3, 3),strides=(2,2),padding = 'valid')(X)
    X = concatenate([mixed_6a1,mixed_6a2,mixed_6a3], axis=-1)
    
    #block mixed_6b
    mixed_6b1 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_6b2 = Conv2D_BN_AC(X, 128, (1, 1), (1, 1), 'same', 'relu')
    mixed_6b2 = Conv2D_BN_AC(mixed_6b2, 128, (1, 7), (1, 1), 'same', 'relu')
    mixed_6b2 = Conv2D_BN_AC(mixed_6b2, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6b3 = Conv2D_BN_AC(X, 128, (1, 1), (1, 1), 'same', 'relu')
    mixed_6b3 = Conv2D_BN_AC(mixed_6b3, 128, (1, 7), (1, 1), 'same', 'relu')
    mixed_6b3 = Conv2D_BN_AC(mixed_6b3, 128, (7, 1), (1, 1), 'same', 'relu')
    mixed_6b3 = Conv2D_BN_AC(mixed_6b3, 128, (1, 7), (1, 1), 'same', 'relu')
    mixed_6b3 = Conv2D_BN_AC(mixed_6b3, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6b4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_6b4 = Conv2D_BN_AC(mixed_6b4, 192, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_6b1,mixed_6b2,mixed_6b3,mixed_6b4], axis=-1)
    
    #block mixed_6c
    mixed_6c1 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_6c2 = Conv2D_BN_AC(X, 160, (1, 1), (1, 1), 'same', 'relu')
    mixed_6c2 = Conv2D_BN_AC(mixed_6c2, 160, (1, 7), (1, 1), 'same', 'relu')
    mixed_6c2 = Conv2D_BN_AC(mixed_6c2, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6c3 = Conv2D_BN_AC(X, 160, (1, 1), (1, 1), 'same', 'relu')
    mixed_6c3 = Conv2D_BN_AC(mixed_6c3, 160, (1, 7), (1, 1), 'same', 'relu')
    mixed_6c3 = Conv2D_BN_AC(mixed_6c3, 160, (7, 1), (1, 1), 'same', 'relu')
    mixed_6c3 = Conv2D_BN_AC(mixed_6c3, 160, (1, 7), (1, 1), 'same', 'relu')
    mixed_6c3 = Conv2D_BN_AC(mixed_6c3, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6c4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_6c4 = Conv2D_BN_AC(mixed_6c4, 192, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_6c1,mixed_6c2,mixed_6c3,mixed_6c4], axis=-1)
    
    #block mixed_6d
    mixed_6d1 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_6d2 = Conv2D_BN_AC(X, 160, (1, 1), (1, 1), 'same', 'relu')
    mixed_6d2 = Conv2D_BN_AC(mixed_6d2, 160, (1, 7), (1, 1), 'same', 'relu')
    mixed_6d2 = Conv2D_BN_AC(mixed_6d2, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(X, 160, (1, 1), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 160, (1, 7), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 160, (7, 1), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 160, (1, 7), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6d4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_6d4 = Conv2D_BN_AC(mixed_6d4, 192, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_6d1,mixed_6d2,mixed_6d3,mixed_6d4], axis=-1)
    
    #block mixed_6e
    mixed_6e1 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_6e2 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_6e2 = Conv2D_BN_AC(mixed_6d2, 192, (1, 7), (1, 1), 'same', 'relu')
    mixed_6e2 = Conv2D_BN_AC(mixed_6d2, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6e3 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 192, (1, 7), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 192, (1, 7), (1, 1), 'same', 'relu')
    mixed_6d3 = Conv2D_BN_AC(mixed_6d3, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_6d4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_6d4 = Conv2D_BN_AC(mixed_6d4, 192, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_6d1,mixed_6d2,mixed_6d3,mixed_6d4], axis=-1)
    
    #block mixed_7a
    mixed_7a1 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_7a1 = Conv2D_BN_AC(mixed_7a1, 320, (3, 3), (2, 2), 'valid', 'relu')
    mixed_7a2 = Conv2D_BN_AC(X, 192, (1, 1), (1, 1), 'same', 'relu')
    mixed_7a2 = Conv2D_BN_AC(mixed_7a2, 192, (1, 7), (1, 1), 'same', 'relu')
    mixed_7a2 = Conv2D_BN_AC(mixed_7a2, 192, (7, 1), (1, 1), 'same', 'relu')
    mixed_7a2 = Conv2D_BN_AC(mixed_7a2, 192, (3, 3), (2, 2), 'valid', 'relu')
    mixed_7a3 = MaxPooling2D((3, 3),strides=(2,2),padding = 'valid')(X)
    X = concatenate([mixed_7a1,mixed_7a2,mixed_7a3], axis=-1)
    
    #block mixed_7b
    mixed_7b1 = Conv2D_BN_AC(X, 320, (1, 1), (1, 1), 'same', 'relu')
    mixed_7b2 = Conv2D_BN_AC(X, 384, (1, 1), (1, 1), 'same', 'relu')
    mixed_7b2 = concatenate([Conv2D_BN_AC(mixed_7b2, 384, (1, 3), (1, 1), 'same', 'relu'), Conv2D_BN_AC(mixed_7b2, 384, (3, 1), (1, 1), 'same', 'relu')], axis=-1)
    mixed_7b3 = Conv2D_BN_AC(X, 448, (1, 1), (1, 1), 'same', 'relu')
    mixed_7b3 = Conv2D_BN_AC(mixed_7b3, 384, (3, 3), (1, 1), 'same', 'relu')
    mixed_7b3 = concatenate([Conv2D_BN_AC(mixed_7b3, 384, (1, 3), (1, 1), 'same', 'relu'), Conv2D_BN_AC(mixed_7b3, 384, (3, 1), (1, 1), 'same', 'relu')], axis=-1)
    mixed_7b4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_7b4 = Conv2D_BN_AC(mixed_7b4, 192, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_7b1,mixed_7b2,mixed_7b3,mixed_7b4], axis=-1)
    
    #block mixed_7c
    mixed_7c1 = Conv2D_BN_AC(X, 320, (1, 1), (1, 1), 'same', 'relu')
    mixed_7c2 = Conv2D_BN_AC(X, 384, (1, 1), (1, 1), 'same', 'relu')
    mixed_7c2 = concatenate([Conv2D_BN_AC(mixed_7c2, 384, (1, 3), (1, 1), 'same', 'relu'), Conv2D_BN_AC(mixed_7c2, 384, (3, 1), (1, 1), 'same', 'relu')], axis=-1)
    mixed_7c3 = Conv2D_BN_AC(X, 448, (1, 1), (1, 1), 'same', 'relu')
    mixed_7c3 = Conv2D_BN_AC(mixed_7c3, 384, (3, 3), (1, 1), 'same', 'relu')
    mixed_7c3 = concatenate([Conv2D_BN_AC(mixed_7c3, 384, (1, 3), (1, 1), 'same', 'relu'), Conv2D_BN_AC(mixed_7c3, 384, (3, 1), (1, 1), 'same', 'relu')], axis=-1)
    mixed_7c4 = AveragePooling2D((3, 3),strides=(1,1),padding = 'same')(X)
    mixed_7c4 = Conv2D_BN_AC(mixed_7c4, 192, (1, 1), (1, 1), 'same', 'relu')
    X = concatenate([mixed_7c1,mixed_7c2,mixed_7c3,mixed_7c4], axis=-1)
    
    X = AveragePooling2D((8, 8), strides=(1, 1))(X)
    X = Dropout(0.8)(X)
    X = Flatten()(X)
    X = Dense(2, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='Inception_v3')
    return model

In [11]:
model = Inception_v3()

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_273 (Conv2D)             (None, 149, 149, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_273 (BatchN (None, 149, 149, 32) 128         conv2d_273[0][0]                 
__________________________________________________________________________________________________
activation_273 (Activation)     (None, 149, 149, 32) 0           batch_normalization_273[0][0]    
__________________________________________________________________________________________________
conv2d_274

In [13]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,epochs = 200, batch_size=32,validation_data=(x_test,y_test), shuffle = True)